In [2]:
import numpy as np
from scipy.stats import norm

def calculate_npv_and_option_value(
    initial_investment=8500000000,  # Cost of nuclear power plant in AUD (Source: recent reports)
    base_cash_flow=1200000000,  # Initial annual revenue (Estimate)
    discount_rate=0.07,  # Discount rate (Source: typical range for nuclear projects)
    volatility=0.25,  # Volatility (Source: general market uncertainties)
    time_to_maturity=25,  # Time in years (2024 to 2050 for net zero target)
    growth_rate=0.02,  # Annual growth rate of cash flows (Estimate)
    decommissioning_cost=900000000  # Estimated decommissioning cost in AUD (Source: typical range)
):
    """
    Calculate NPV and real option value for nuclear power plant investment
    with enhanced error checking and documentation.
    """
    # Input validation
    if initial_investment <= 0 or base_cash_flow <= 0:
        raise ValueError("Investment and cash flow must be positive")
    if not (0 < discount_rate < 1) or not (0 < volatility < 1):
        raise ValueError("Discount rate and volatility must be between 0 and 1")
    
    # Generate cash flows with more realistic assumptions
    cash_flows = []
    for year in range(time_to_maturity):
        # Ramp-up period in first 3 years
        ramp_up_factor = min(1.0, (year + 1) / 3)
        annual_cf = base_cash_flow * (1 + growth_rate)**year * ramp_up_factor
        
        # Add maintenance costs (typically 2-3% of initial investment annually)
        maintenance_cost = initial_investment * 0.025
        annual_cf -= maintenance_cost
        
        cash_flows.append(annual_cf)
    
    # Add decommissioning cost at the end
    cash_flows[-1] -= decommissioning_cost
    
    # Present Value calculation
    pv_cash_flows = sum(cf / (1 + discount_rate) ** t 
                       for t, cf in enumerate(cash_flows, start=1))
    
    # Black-Scholes calculation with error handling
    try:
        d1 = (np.log(pv_cash_flows / initial_investment) + 
              (discount_rate + 0.5 * volatility ** 2) * time_to_maturity) / \
             (volatility * np.sqrt(time_to_maturity))
        d2 = d1 - volatility * np.sqrt(time_to_maturity)
        
        option_value = pv_cash_flows * norm.cdf(d1) - \
                      initial_investment * np.exp(-discount_rate * time_to_maturity) * \
                      norm.cdf(d2)
    except ValueError as e:
        raise ValueError(f"Error in Black-Scholes calculation: {e}")
    
    return pv_cash_flows, option_value, cash_flows

def run_scenario_analysis(base_pv, base_option, initial_investment, 
                         time_to_maturity, discount_rate, volatility):
    """
    Run enhanced scenario analysis with more realistic utilisation rates
    and additional scenarios.
    """
    scenarios = {
        "Low Nuclear Adoption (10% average utilisation)": 0.10,
        "Medium Nuclear Adoption (20% average utilisation)": 0.20,
        "High Nuclear Adoption (40% average utilisation)": 0.40,
        "Optimal Nuclear Adoption (90% average utilisation)": 0.90
    }
    
    results = {}
    for scenario, utilisation_rate in scenarios.items():
        pv_adjusted = base_pv * utilisation_rate
        
        # Recalculate option value with adjusted PV
        d1 = (np.log(pv_adjusted / initial_investment) + 
              (discount_rate + 0.5 * volatility ** 2) * time_to_maturity) / \
             (volatility * np.sqrt(time_to_maturity))
        d2 = d1 - volatility * np.sqrt(time_to_maturity)
        
        option_value = pv_adjusted * norm.cdf(d1) - \
                      initial_investment * np.exp(-discount_rate * time_to_maturity) * \
                      norm.cdf(d2)
                      
        results[scenario] = (pv_adjusted, option_value)
    
    return results

# Calculate base case
pv_cash_flows, option_value, cash_flows = calculate_npv_and_option_value()

# Run enhanced scenario analysis
scenario_results = run_scenario_analysis(
    pv_cash_flows, option_value,
    initial_investment=8500000000,
    time_to_maturity=25,
    discount_rate=0.07,
    volatility=0.25
)

# Print results with improved formatting
print(f"\nBase Case Analysis:")
print(f"Present Value of Expected Cash Flows: AUD {pv_cash_flows:,.2f}")
print(f"Option Value of Delaying Investment: AUD {option_value:,.2f}")
print(f"Net Present Value (NPV): AUD {pv_cash_flows - 8500000000:,.2f}")

print(f"\nScenario Analysis:")
for scenario, (pv, opt_value) in scenario_results.items():
    print(f"\n{scenario}")
    print(f"Present Value: AUD {pv:,.2f}")
    print(f"Option Value: AUD {opt_value:,.2f}")
    print(f"NPV: AUD {pv - 8500000000:,.2f}")


Base Case Analysis:
Present Value of Expected Cash Flows: AUD 12,999,035,439.48
Option Value of Delaying Investment: AUD 11,600,372,593.65
Net Present Value (NPV): AUD 4,499,035,439.48

Scenario Analysis:

Low Nuclear Adoption (10% average utilisation)
Present Value: AUD 1,299,903,543.95
Option Value: AUD 564,234,565.48
NPV: AUD -7,200,096,456.05

Medium Nuclear Adoption (20% average utilisation)
Present Value: AUD 2,599,807,087.90
Option Value: AUD 1,596,807,978.25
NPV: AUD -5,900,192,912.10

High Nuclear Adoption (40% average utilisation)
Present Value: AUD 5,199,614,175.79
Option Value: AUD 3,974,384,108.43
NPV: AUD -3,300,385,824.21

Optimal Nuclear Adoption (90% average utilisation)
Present Value: AUD 11,699,131,895.53
Option Value: AUD 10,313,593,064.38
NPV: AUD 3,199,131,895.53
